In [1]:
import QuantLib as ql
from scipy.interpolate import interp1d
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Set display options for better readability
pd.options.display.float_format = "{:,.2f}".format
plt.style.use('seaborn-v0_8')

# Market Data Setup

In [3]:

t = np.array([0, 0.3, 0.6, 0.9, 1.2, 1.5, 1.8, 2.1, 2.4, 2.7, 3.0, 3.3])
zero_rates_base = np.array([0.0, 0.0673, 0.0855, 0.0926, 0.0971, 0.1005,
                            0.1035, 0.1063, 0.1089, 0.1115, 0.1142, 0.1169])

# Set up calendar and trade dates
calendar = ql.WeekendsOnly()
trade_date = ql.Date(15, 11, 2023)
maturity_date = ql.Date(15, 11, 2026)
ql.Settings.instance().evaluationDate = trade_date

# Swap contract details
notional = 1_000_000
fixed_rate = 0.1141  # 11.41%
accrual_conv = 365  # day count convention
period = ql.Period("3M")  # quarterly payments

# Swap Valuation Function

In [5]:
def value_payer_swap(zero_rates, scenario_name):
   
    discount_factor = np.exp(-zero_rates * t)
    
    
    schedule = ql.MakeSchedule(trade_date, maturity_date, period,
                               calendar=calendar, convention=ql.Following)
    dates = list(schedule)
    year_frac = [(d - trade_date) / accrual_conv for d in dates]
    
    
    interp = interp1d(t, discount_factor, kind='linear', fill_value="extrapolate")
    interp_df = np.array([float(interp(y)) for y in year_frac])
    
 
    forward = [0.0]
    for i in range(1, len(dates)):
        dt = year_frac[i] - year_frac[i-1]
        fwd = (interp_df[i-1] / interp_df[i] - 1.0) / dt if interp_df[i] > 0 and dt > 0 else 0.0
        forward.append(fwd)
    
    # Calculate cash flows
    fixed_cf, float_cf = [0.0], [0.0]
    for i in range(1, len(dates)):
        dt = year_frac[i] - year_frac[i-1]
        fixed_cf.append(notional * fixed_rate * dt)
        float_cf.append(notional * forward[i] * dt)
    
    # Calculate present values
    pv_fixed = float(np.sum(np.array(fixed_cf) * interp_df))
    pv_float = float(np.sum(np.array(float_cf) * interp_df))
    pv_swap_payer = pv_float - pv_fixed
    
    # cash flow table
    cf_table = pd.DataFrame({
        "Date": dates,
        "YearFrac": year_frac,
        "DF": interp_df,
        "Forward": forward,
        "FixedCF": fixed_cf,
        "FloatCF": float_cf,
        "PV_FixedCF": np.array(fixed_cf) * interp_df,
        "PV_FloatCF": np.array(float_cf) * interp_df
    })
    
    return pv_fixed, pv_float, pv_swap_payer, cf_table

# Base Case Valuation

In [6]:
# Base case valuation
pv_fixed_base, pv_float_base, pv_swap_base, cf_base = value_payer_swap(zero_rates_base, "Base")

print("Swap valuation assumptions:")
print(f"PAYER swap (pay fixed @ {fixed_rate:.2%}, receive float)")
print(f"Notional = {notional:,}")
print(f"Trade date: {trade_date}")
print(f"Maturity date: {maturity_date}")
print(f"Day count: Actual/{accrual_conv}")
print(f"Payment frequency: {period}")
print("\nBase Scenario Results:")
print(f"PV Fixed Leg: {pv_fixed_base:,.2f}")
print(f"PV Floating Leg: {pv_float_base:,.2f}") 
print(f"Swap NPV: {pv_swap_base:,.2f}")

Swap valuation assumptions:
PAYER swap (pay fixed @ 11.41%, receive float)
Notional = 1,000,000
Trade date: November 15th, 2023
Maturity date: November 15th, 2026
Day count: Actual/365
Payment frequency: 3M

Base Scenario Results:
PV Fixed Leg: 290,480.57
PV Floating Leg: 290,625.73
Swap NPV: 145.17


# Stress Testing Scenarios

## 1. Parallel shift down (-50 bps)

In [8]:

results_list = []

# Add base case to results
results_list.append({
    "Scenario": "Base",
    "PV_Fixed": pv_fixed_base,
    "PV_Float": pv_float_base,
    "PV_Swap": pv_swap_base
})

# 1. Parallel shift down (-50 bps)
shift = -0.005
zero_rates_parallel_down = zero_rates_base + shift
pv_fixed_par_down, pv_float_par_down, pv_swap_par_down, _ = value_payer_swap(zero_rates_parallel_down, "Parallel Shift Down")
results_list.append({
    "Scenario": f"Parallel Shift {shift:+.2%}",
    "PV_Fixed": pv_fixed_par_down,
    "PV_Float": pv_float_par_down,
    "PV_Swap": pv_swap_par_down
})



## 2. Parallel shift up (+50 bps)

In [9]:

shift = 0.005
zero_rates_parallel_up = zero_rates_base + shift
pv_fixed_par_up, pv_float_par_up, pv_swap_par_up, _ = value_payer_swap(zero_rates_parallel_up, "Parallel Shift Up")
results_list.append({
    "Scenario": f"Parallel Shift {shift:+.2%}",
    "PV_Fixed": pv_fixed_par_up,
    "PV_Float": pv_float_par_up,
    "PV_Swap": pv_swap_par_up
})



## 3. Steepener (short rates down, long rates up)

In [11]:

zero_rates_steepener = zero_rates_base.copy()
zero_rates_steepener[:4] -= 0.01  # Short end down 100 bps
zero_rates_steepener[-4:] += 0.01  # Long end up 100 bps
pv_fixed_steep, pv_float_steep, pv_swap_steep, _ = value_payer_swap(zero_rates_steepener, "Steepener")
results_list.append({
    "Scenario": "Steepener (Short↓100bps, Long↑100bps)",
    "PV_Fixed": pv_fixed_steep,
    "PV_Float": pv_float_steep,
    "PV_Swap": pv_swap_steep
})



## 4. Flattener (short rates up, long rates down)

In [13]:

zero_rates_flattener = zero_rates_base.copy()
zero_rates_flattener[:4] += 0.01  # Short end up 100 bps
zero_rates_flattener[-4:] -= 0.01  # Long end down 100 bps
pv_fixed_flat, pv_float_flat, pv_swap_flat, _ = value_payer_swap(zero_rates_flattener, "Flattener")
results_list.append({
    "Scenario": "Flattener (Short↑100bps, Long↓100bps)",
    "PV_Fixed": pv_fixed_flat,
    "PV_Float": pv_float_flat,
    "PV_Swap": pv_swap_flat
})



## 5. Short end shock (first 4 points up 75 bps)

In [14]:

zero_rates_short = zero_rates_base.copy()
zero_rates_short[:4] += 0.0075
pv_fixed_short, pv_float_short, pv_swap_short, _ = value_payer_swap(zero_rates_short, "Short End Shock")
results_list.append({
    "Scenario": "Short End Shock (+75bps)",
    "PV_Fixed": pv_fixed_short,
    "PV_Float": pv_float_short,
    "PV_Swap": pv_swap_short
})



## 6. Long end shock (last 4 points up 75 bps)

In [15]:

zero_rates_long = zero_rates_base.copy()
zero_rates_long[-4:] += 0.0075
pv_fixed_long, pv_float_long, pv_swap_long, _ = value_payer_swap(zero_rates_long, "Long End Shock")
results_list.append({
    "Scenario": "Long End Shock (+75bps)",
    "PV_Fixed": pv_fixed_long,
    "PV_Float": pv_float_long,
    "PV_Swap": pv_swap_long
})



## 7. Random shock (σ=20bps)

In [16]:

np.random.seed(42)
rand_noise = np.random.normal(0.0, 0.002, size=zero_rates_base.shape)
zero_rates_random = zero_rates_base + rand_noise
pv_fixed_rand, pv_float_rand, pv_swap_rand, _ = value_payer_swap(zero_rates_random, "Random Shock")
results_list.append({
    "Scenario": "Random Shock (σ=20bps)",
    "PV_Fixed": pv_fixed_rand,
    "PV_Float": pv_float_rand,
    "PV_Swap": pv_swap_rand
})

# Convert results to DataFrame
results = pd.DataFrame(results_list)
results["Diff_vs_Base"] = results["PV_Swap"] - pv_swap_base
results["PctDiff_vs_Base"] = results["Diff_vs_Base"] / abs(pv_swap_base) * 100

print("\n--- Stress Test Results ---")
print(results.to_string(index=False))


--- Stress Test Results ---
                             Scenario   PV_Fixed   PV_Float    PV_Swap  Diff_vs_Base  PctDiff_vs_Base
                                 Base 290,480.57 290,625.73     145.17          0.00             0.00
                Parallel Shift -0.50% 292,726.13 279,885.99 -12,840.14    -12,985.31        -8,945.05
                Parallel Shift +0.50% 288,257.67 301,205.28  12,947.61     12,802.44         8,819.08
Steepener (Short↓100bps, Long↑100bps) 289,073.57 311,627.01  22,553.44     22,408.27        15,436.14
Flattener (Short↑100bps, Long↓100bps) 291,945.80 268,983.62 -22,962.17    -23,107.34       -15,917.70
             Short End Shock (+75bps) 290,059.45 290,625.73     566.29        421.12           290.09
              Long End Shock (+75bps) 288,996.62 306,435.72  17,439.10     17,293.93        11,913.08
               Random Shock (σ=20bps) 290,225.74 288,646.92  -1,578.83     -1,723.99        -1,187.59
